In [1]:
import gym
import torch
import torch.nn as nn
from itertools import count
from torch.distributions import Normal
import numpy as np
from collections import deque
import random
import torch.nn.functional as F
from tensorboardX import SummaryWriter


import gym
import network_sim
from gym import spaces
import numpy as np
import random


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
class PolicyNetwork(nn.Module):
    def __init__(self):
        super(PolicyNetwork, self).__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(30, 64)
        self.fc2 = nn.Linear(64, 256)
        self.fc_mu = nn.Linear(256, 1)
        self.fc_std = nn.Linear(256, 1)
        self.tanh = nn.Tanh()
        self.softplus = nn.Softplus()
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        mu = 2 * self.tanh(self.fc_mu(x))
        std = self.softplus(self.fc_std(x)) + 1e-3
        return mu, std
    def select_action(self, state):
        with torch.no_grad():
            mu, std = self.forward(state)
            n = Normal(mu, std)
            action = n.sample()
        
        return np.clip(action.item(), -1e12, 1e12)
            
        
#         return np.clip(action.item(), -2., 2.)

In [4]:
class ValueNetwork(nn.Module):
    def __init__(self):
        super(ValueNetwork, self).__init__()
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(30, 64)
        self.fc2 = nn.Linear(64, 256)
        self.fc3 = nn.Linear(256, 1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
class Memory(object):
    def __init__(self, memory_size: int) -> None:
        self.memory_size = memory_size
        self.buffer = deque(maxlen=self.memory_size)

    def add(self, experience) -> None:
        self.buffer.append(experience)

    def size(self):
        return len(self.buffer)

    def sample(self, batch_size: int, continuous: bool = True):
        if batch_size > len(self.buffer):
            batch_size = len(self.buffer)
        if continuous:
            rand = random.randint(0, len(self.buffer) - batch_size)
            return [self.buffer[i] for i in range(rand, rand + batch_size)]
        else:
            indexes = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
            return [self.buffer[i] for i in indexes]

    def clear(self):
        self.buffer.clear()

In [6]:
env = gym.make('PccNs-v0')
policy = PolicyNetwork().to(device)
value = ValueNetwork().to(device)
optim = torch.optim.Adam(policy.parameters(), lr=1e-3)
value_optim = torch.optim.Adam(value.parameters(), lr=1e-3)
gamma = 0.99
writer = SummaryWriter('a2c_logs_pen')
memory = Memory(32)
batch_size = 32
is_learn = False
steps = 0

History length: 10
Features: ['sent latency inflation', 'latency ratio', 'send ratio']
Getting min obs for ['sent latency inflation', 'latency ratio', 'send ratio']


/home/kun/anaconda3/envs/CVpy3/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [7]:
for epoch in count():
    state = env.reset()
    episode_reward = 0
    k = 0
    for time_steps in range(200):
        k += 1
        state_tensor = torch.FloatTensor(state).unsqueeze(0).to(device)
        action = policy.select_action(state_tensor)
        next_state, reward, done, _ = env.step([action])
        episode_reward += reward

        reward = (reward + 8.1) / 8.1
        memory.add((state, next_state, action, reward, done))
        if k == 32 or time_steps == 199:
            k = 0
            steps += 1
            experiences = memory.sample(batch_size)
            batch_state, batch_next_state, batch_action, batch_reward, batch_done = zip(*experiences)
            batch_state = torch.FloatTensor(batch_state).to(device)
            batch_next_state = torch.FloatTensor(batch_next_state).to(device)
            batch_action = torch.FloatTensor(batch_action).unsqueeze(1).to(device)
            batch_reward = torch.FloatTensor(batch_reward).unsqueeze(1).to(device)
            batch_done = torch.FloatTensor(batch_done).unsqueeze(1).to(device)

            # print(batch_state.shape, batch_next_state.shape, batch_action.shape, batch_reward.shape)

            with torch.no_grad():
                value_target = batch_reward + gamma * (1 - batch_done) * value(batch_next_state)
                advantage = value_target - value(batch_state)
            mu, std = policy(batch_state)
            # print(prob.shape)
            n = Normal(mu, std)
            log_prob = n.log_prob(batch_action)
            loss = - log_prob * advantage
            # print(loss.shape)
            loss = loss.mean()
            optim.zero_grad()
            loss.backward()
            optim.step()
            writer.add_scalar('action loss', loss.item(), steps)

            value_loss = F.mse_loss(value_target, value(batch_state))
            value_optim.zero_grad()
            value_loss.backward()
            value_optim.step()
            writer.add_scalar('value loss', value_loss.item(), steps)

            memory.clear()
        if done:
            break
        state = next_state
    writer.add_scalar('episode reward', episode_reward, epoch)
    if epoch % 10 == 0:
        print('Epoch:{}, episode reward is {}'.format(epoch, episode_reward))
        torch.save(policy.state_dict(), 'a2c-pen-policy.para')

Reward: 0.00, Ewma Reward: 0.00
Epoch:0, episode reward is 146.8920976195483
Reward: 146.89, Ewma Reward: 1.47
Reward: -4.06, Ewma Reward: 1.41
Reward: 26.46, Ewma Reward: 1.66
Reward: -28.47, Ewma Reward: 1.36
Reward: -36.27, Ewma Reward: 0.99
Reward: -72.05, Ewma Reward: 0.26


KeyboardInterrupt: 